# Métrica 2_2_04 — Congruencia entre dependientes económicos con actividad laboral e inversiones declaradas

| Escenario                                                                                            | Condición      | Resultado        |
| ---------------------------------------------------------------------------------------------------- | -------------- | ---------------- |
| No existen dependientes ni inversiones (y no es obligatorio)                                         | —              | ⚪ **SIN_DATO**   |
| Hay dependiente con actividad laboral + inversión titular “dependiente” o “declarante y dependiente” | ✅              | 🟢 **CUMPLE**    |
| Hay dependiente con actividad laboral pero sin inversiones compartidas                               | ❌              | 🔴 **NO_CUMPLE** |
| No hay dependiente con actividad laboral                                                             | ⚪ **SIN_DATO** |                  |


In [1]:
import traceback
from pymongo import MongoClient, UpdateOne
from config import MONGO_URI, DB_NAME, SOURCE_COLLECTION_NAME, METRICS_COLLECTION_NAME

METRIC_ID = "2_2_04_CONGRUENCIA_DEPENDIENTES_INVERSIONES"

DICT_INFO = {
    "dependientes_path": "declaracion.situacionPatrimonial.datosDependientesEconomicos.dependienteEconomico",
    "inversiones_path": "declaracion.situacionPatrimonial.inversionesCuentasValores.inversion",
    "obligatorio": False
}

# --- Funciones auxiliares ---

def extraer_lista(doc, path):
    try:
        partes = path.split(".")
        actual = doc
        for p in partes:
            if isinstance(actual, dict):
                actual = actual.get(p, {})
            else:
                return []
        if isinstance(actual, list):
            return actual
        elif isinstance(actual, dict):
            return [actual]
        return []
    except Exception:
        return []

def dependiente_tiene_actividad(dep):
    try:
        act = dep.get("actividadLaboral")
        if isinstance(act, dict):
            return any(v for v in act.values() if v)
        if isinstance(act, str):
            return act.strip() != ""
        return False
    except Exception:
        return False

def existen_dependientes_activos(dependientes):
    if not isinstance(dependientes, list):
        return False
    return any(dependiente_tiene_actividad(dep) for dep in dependientes)

def hay_inversion_con_dependiente(inversiones):
    for inv in inversiones:
        titulares = inv.get("titular", [])
        if isinstance(titulares, dict):
            titulares = [titulares]
        for t in titulares:
            valor = str(t.get("valor", "")).upper()
            if "DEPENDIENTE" in valor or "DECLARANTE Y DEPENDIENTE" in valor:
                return True
    return False

# --- Evaluación principal ---

def evaluar_metrica(doc):
    dependientes = extraer_lista(doc, DICT_INFO["dependientes_path"])
    inversiones = extraer_lista(doc, DICT_INFO["inversiones_path"])

    tiene_dependiente_activo = existen_dependientes_activos(dependientes)

    # No hay dependientes ni inversiones
    if not tiene_dependiente_activo and not inversiones and not DICT_INFO["obligatorio"]:
        return "SIN_DATO"

    # Hay dependientes con actividad laboral y alguna inversión compartida
    if tiene_dependiente_activo and hay_inversion_con_dependiente(inversiones):
        return "CUMPLE"

    # Hay dependientes con actividad laboral pero sin inversiones compartidas
    if tiene_dependiente_activo and not hay_inversion_con_dependiente(inversiones):
        return "NO_CUMPLE"

    # Default
    return "SIN_DATO"

# --- Procesamiento MongoDB ---

def procesar_metrica_2_2_04():
    resultados = {"CUMPLE": 0, "NO_CUMPLE": 0, "SIN_DATO": 0}
    operaciones = []
    total = 0

    try:
        print(f"Procesando métrica {METRIC_ID}...")
        client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
        db = client[DB_NAME]
        src = db[SOURCE_COLLECTION_NAME]
        tgt = db[METRICS_COLLECTION_NAME]

        cursor = src.find({}, {
            "_id": 1,
            "declaracion.situacionPatrimonial.datosDependientesEconomicos": 1,
            "declaracion.situacionPatrimonial.inversionesCuentasValores": 1
        }, no_cursor_timeout=True)

        for doc in cursor:
            total += 1
            try:
                resultado = evaluar_metrica(doc)
            except Exception as e:
                resultado = "SIN_DATO"
                print(f"Error en doc {doc.get('_id')}: {e}")

            resultados[resultado] = resultados.get(resultado, 0) + 1
            operaciones.append(UpdateOne({"_id": doc["_id"]}, {"$set": {METRIC_ID: resultado}}, upsert=True))

            if len(operaciones) >= 1000:
                tgt.bulk_write(operaciones)
                operaciones.clear()
                print(f"  > Procesados {total} documentos...")

        if operaciones:
            tgt.bulk_write(operaciones)

        print("\n--- RESUMEN FINAL ---")
        print(f"Total procesados: {total}")
        for k, v in resultados.items():
            print(f"  > {k}: {v}")

    except Exception:
        traceback.print_exc()
    finally:
        try:
            client.close()
        except:
            pass
        print("Conexión cerrada.")


if __name__ == "__main__":
    procesar_metrica_2_2_04()


Procesando métrica 2_2_04_CONGRUENCIA_DEPENDIENTES_INVERSIONES...


c:\Users\Mauro\anaconda3\Lib\site-packages\pymongo\synchronous\collection.py:1954: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://mongodb.com/docs/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)


  > Procesados 1000 documentos...
  > Procesados 2000 documentos...
  > Procesados 3000 documentos...
  > Procesados 4000 documentos...
  > Procesados 5000 documentos...
  > Procesados 6000 documentos...
  > Procesados 7000 documentos...
  > Procesados 8000 documentos...
  > Procesados 9000 documentos...
  > Procesados 10000 documentos...
  > Procesados 11000 documentos...
  > Procesados 12000 documentos...
  > Procesados 13000 documentos...
  > Procesados 14000 documentos...
  > Procesados 15000 documentos...
  > Procesados 16000 documentos...
  > Procesados 17000 documentos...
  > Procesados 18000 documentos...
  > Procesados 19000 documentos...
  > Procesados 20000 documentos...
  > Procesados 21000 documentos...
  > Procesados 22000 documentos...
  > Procesados 23000 documentos...
  > Procesados 24000 documentos...
  > Procesados 25000 documentos...
  > Procesados 26000 documentos...
  > Procesados 27000 documentos...
  > Procesados 28000 documentos...
  > Procesados 29000 document

KeyboardInterrupt: 